In [ ]:
import pandas as pd

Using queries3.sql

1 user action per day up to 5x to get the full payment:

- US, UK, IE, DE, ES, IT: 8.75 USD (5 Daily User Actions to Max Partner Payment per User per month)
- BR: 7.50 USD (5 Daily User Actions to Max Partner Payment per User per month)
- MX, CL, AR: $6.25 USD (5 Daily User Actions to Max Partner Payment per User per month)

Day 1 (any user = 1.75$), then throughout the month, they would need to open the app 4 separate days to get full payment

### Importing data

In [ ]:
OUTPUT_DIR = "/Users/emulie/Data/Wellhub/output"

In [ ]:
df_internals = pd.read_csv("/Users/emulie/Data/Wellhub/wellhub_monthly_users_20250129.csv")
df_wellhub = pd.read_csv("/Users/emulie/Data/Wellhub/wellhub_users.csv")

In [ ]:
df_internals.head()

In [ ]:
df_wellhub.head()

In [ ]:
# 
print(len(df_internals['user_pseudo_id'].unique()))
print(df_internals.shape[0])
print()
print(len(df_wellhub['email'].unique()))
print(df_wellhub.shape[0])

### Cleaning up the data

In [ ]:
# --- only keeping one email
df_internals_clean = df_internals.drop(columns=['user_id', 'user_pseudo_id'])

In [ ]:
is_duplicate_mask = df_internals_clean.duplicated()
df_duplicates = df_internals_clean[is_duplicate_mask]

Dropping duplicates is correct

In [ ]:
df_internals_clean = df_internals_clean.drop_duplicates()

In [ ]:
print(df_internals_clean.shape[0])

### Merging data

In [ ]:
t = pd.merge(df_wellhub, df_internals_clean, on='email', how='inner')
print(f"How many wellhub users used BetterSleep: {t.shape[0]}; Perc: {t.shape[0] / df_wellhub.shape[0] * 100: .2f}%")

In [ ]:
df_merged = pd.merge(df_wellhub, df_internals_clean, on='email', how='left')

In [ ]:
df_merged = df_merged.fillna(0.0)

In [ ]:
print(df_merged['sep2024'].sum())
print(df_merged['oct2024'].sum())
print(df_merged['nov2024'].sum())
print(df_merged['dec2024'].sum())
print(df_merged['jan2025'].sum())

In [ ]:
df_merged.head()

In [ ]:
df_merged.to_csv(f"{OUTPUT_DIR}/usage_20250129.csv")

### Computing $$

In [ ]:
# --- pricing is USD
dct_pricing = {
    'us': 8.75,
    'gb': 8.75, 
    'uk': 8.75,
    'ie': 8.75,
    'de': 8.75, 
    'es': 8.75,
    'it': 8.75,
    'br': 7.50, 
    'mx': 6.25,
    'cl': 6.25,
    'ar': 6.25
}

months = ['sep2024', 'oct2024', 'nov2024', 'dec2024', 'jan2025']

In [ ]:
def get_monthly_fees(country: str, num_usages: int):
    if num_usages > 0 and num_usages < 5:
        return 1.75
    elif num_usages == 0:
        return 0
    else: # num_usage >=5 
        return dct_pricing[country]

In [ ]:
df_money = df_merged.copy()
for month in months:
    df_money[f"{month}_USD"] = df_money.apply(lambda x: get_monthly_fees(x['country_code'], x[month]), axis=1)

In [ ]:
t = df_money[[f"{month}_USD" for month in months]]
df_revenues = t.sum().reset_index().rename(columns={'index': 'month', 0: 'revenues (USD)'})
df_revenues['Monthly Users'] = t.astype(bool).sum(axis=0).values
df_revenues['month'] = months

In [ ]:
df_revenues.head()

In [ ]:
df_revenues.to_csv(f"{OUTPUT_DIR}/revenues_20250129.csv")

In [ ]:
df_revenues['revenues (USD)'].sum()